<a href="https://colab.research.google.com/github/Serena428/Rader_performance_prediction/blob/main/model_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge

from sklearn.ensemble import BaggingRegressor

from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error

# Global settings

warnings.filterwarnings("ignore") # To ignore warnings
n_jobs = -1 # This parameter conrols the parallel processing. -1 means using all processors.
random_state = 42 # This parameter controls the randomness of the data. Using some int value to get same results everytime this code is run.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df=pd.read_csv('/content/drive/MyDrive/수연/study/radar/train.csv')
#train_df.head()
len(df)

39607

In [4]:
df.head()

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [5]:
df.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', 'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

In [6]:
X = df[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56']]

In [7]:
y = df[['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']]

#Train and Test Data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.33, random_state=random_state)

print(f'Training set--> X_train shape= {X_train.shape}, y_train shape= {y_train.shape}')
print(f'Holdout set--> X_test shape= {X_test.shape}, y_test shape= {y_test.shape}')

Training set--> X_train shape= (26536, 56), y_train shape= (26536, 14)
Holdout set--> X_test shape= (13071, 56), y_test shape= (13071, 14)


# Regression

In [9]:
# def lg_nrmse(gt, preds):
#     # 각 Y Feature별 NRMSE 총합
#     # Y_01 ~ Y_08 까지 20% 가중치 부여
#     all_nrmse = []
#     for idx in range(1,15): # ignore 'ID'
#         rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
#         nrmse = rmse/np.mean(np.abs(gt[:,idx]))
#         all_nrmse.append(nrmse)
#     score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
#     return score

In [10]:
models_scores = [] # To store model scores

def rmse(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared= False) # squared= False > returns Root Mean Square Error 

##Linear Regression

해당 데이터가 선형성을 띄지 않기 때문에 좋은 결과가 나오지는 못할것 같음.

In [11]:
linear_regression = make_pipeline(LinearRegression())
score = rmse(linear_regression)

models_scores.append(['Linear Regression', score])
print(f'LinearRegression Score={score}')

LinearRegression Score=0.9707811126657843


#Lasso Regression

In [12]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.005,
                                            random_state=random_state))

score = rmse(lasso)
models_scores.append(['Lasso', score])
print(f'Lasso Score = {score}')

Lasso Score = 0.971810201221009


#ElasticNet Regression

In [13]:
elastic_net = make_pipeline(RobustScaler(), ElasticNet(alpha=.0005,
                                                        l1_ratio= .9,
                                                        random_state=random_state))

score = rmse(elastic_net)
models_scores.append(['ElasticNet', score])
print(f'ElasticNet Score={score}')

ElasticNet Score=0.9706706830198836


#KernelRidge Regression

In [14]:
# kernel_ridge= KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
# score = rmse(kernel_ridge)
# models_scores.append(['KernelRidge', score])
# print(f'KernelRidge Score= {score}')

In [15]:
# Ranking the scores of each model
pd.DataFrame(models_scores).sort_values(by=[1], ascending=False)

,0,1
1,Lasso,0.971810
0,Linear Regression,0.970781
2,ElasticNet,0.970671


#Ensemble Modeling

In [16]:
def begging_predictions(estimator):
  regr = BaggingRegressor(base_estimator=estimator,
                          n_estimators=10,
                          max_samples =1.0,
                          bootstrap=True,
                          n_jobs=n_jobs,
                          random_state=random_state).fit(X_train,y_train)
  br_y_pred=regr.predict(X_test)

  rmse_val = mean_squared_error(y_test, br_y_pred, squared=False) #squared=False > returns Root Mean square Error

  print(f'RMSE for base estimator {regr.base_estimator_}={rmse_val}\n')
  return br_y_pred

In [19]:
predictions = np.column_stack((begging_predictions(linear_regression),
                               begging_predictions(lasso),
                               begging_predictions(elastic_net)
                               #begging_predictions(kernel_ridge)
                               ))
print(f'bagged predictions shape:{predictions.shape}')

RMSE for base estimator Pipeline(steps=[('linearregression', LinearRegression())])=0.9708333495215723

RMSE for base estimator Pipeline(steps=[('robustscaler', RobustScaler()),
                ('lasso', Lasso(alpha=0.005, random_state=42))])=0.971632328094954

RMSE for base estimator Pipeline(steps=[('robustscaler', RobustScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.0005, l1_ratio=0.9, random_state=42))])=0.9707081052213155

bagged predictions shape:(13071, 42)
